In [1]:
# !pip install pytorch
# !pip install transformers
#! sudo apt install tesseract-ocr
#! pip install pytesseract
#! pip install transformers datasets

In [1]:
from transformers import BertModel
from transformers import BertTokenizer
import torch


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

import os
import numpy as np
np.random.seed(42)
# tqdm.tqdm(dirs, desc='dirs') 
from tqdm import tqdm
import pandas as pd
import itertools
from PIL import Image
import pytesseract
import re
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer #tokanization and removel of punctuation 

import warnings
warnings.filterwarnings('ignore')
import random

import pandas as pd
import numpy as np
import tensorflow as tf

# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Layer#, InputSpec
from tensorflow.keras import initializers, regularizers, constraints
#from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Input, Dense, GRU, Bidirectional, TimeDistributed, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

#from sklearn.metrics import roc_auc_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#classes
txt_path='/media/umar_visionx/Backup Plus/Active/Faizan/doc_classifier_classes.txt'
with open(txt_path,'r') as file:
    classes=file.read().split('\n')
print(classes)

['Admin Note', 'Clinical History And Summary', 'Consult Note', 'Cover Page', 'Imaging Note', 'Insurance Authorization', 'Intake Forms', 'Lab Test', 'Other', 'Patient Profile', 'Prescriptions', 'Referral Letter', 'Requisition Form']


In [5]:
#to extract text from the document we are applying OCR to each document
def apply_ocr(sample):

        # apply ocr to the image
        #Extracting words and bounding boxes from images
        ocr_dframe = pytesseract.image_to_data(sample, output_type='data.frame')
        #return a subset of the DataFrame's columns based on the column dtypes.
        float_cols = ocr_dframe.select_dtypes('float').columns
        #dropping the rows where atleast one element is not present(i.e "text" is not present)
        ocr_dframe = ocr_dframe.dropna().reset_index(drop=True)
        #selecting float_cols,round along zero,changing type to integer
        ocr_dframe[float_cols] = ocr_dframe[float_cols].round(0).astype(int)
        #replace field that's entirely space (or empty) with NaN check https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
        ocr_dframe = ocr_dframe.replace(r'^\s*$', np.nan, regex=True)
        #dropping the rows where atleast one element is not present(i.e "text" is not present)
        ocr_dframe = ocr_dframe.dropna().reset_index(drop=True)

        # get the words and actual (unnormalized) bounding boxes
        #words = [word for word in ocr_dframe.text if str(word) != 'nan'])
        words = list(ocr_dframe.text)#making the list of column "text" in 'ocr_dframe' dataframe
        #words = [str(w) for w in words] #converting into list of string of words from the last words list 

        #making a list of words in the image
        image_words=[]
        for word in words:
          if str(word)!="nan":
            image_words.append(str(word))

      
        #joining words with space
        words= " ".join(image_words)
        return words

In [6]:
#making folders and saving the text files (with extracted text from image) in related folders
def extract_text(source_path,dest_path):
    #first making folders
    try:
        os.makedirs(dest_path)
        for ent in classes:
            folder_path=os.path.join(dest_path,ent)
            os.makedirs(folder_path)
            print("Folder making process completed")
    except:
        pass
    for folder in os.listdir(source_path):
        folder_path=os.path.join(source_path,folder)
        for files in os.listdir(folder_path):
            f_path=os.path.join(folder_path,files)#path of image
            image = Image.open(f_path)
            image = image.convert("RGB")
            para=apply_ocr(image) #getting words from image
        
        #making directory to save the files with same file and folder name
            txt_file=os.path.splitext(files)[0] + ".txt" #making same name of file as image
            txt_path=os.path.join(dest_path,os.path.join(folder,txt_file))
        #writing and saving file
            with open(txt_path, "w") as file:
                file.write(para)
    print("All text files are created successfully")

In [7]:
#extracting text from images and creating text files 
image_data_source="/media/umar_visionx/Backup Plus/Active/Faizan/image_data"#image data source path
test_image_source="/media/umar_visionx/Backup Plus/Active/Faizan/test_images"#test image data source path
text_dest="/media/umar_visionx/Backup Plus/Active/Faizan/dataset"#extracted text files destinatio path for image data
test_text_dest="/media/umar_visionx/Backup Plus/Active/Faizan/test_dataset"#extracted text files destination path from test image data

extract_text(image_data_source,text_dest)
extract_text(test_image_source,test_text_dest)

In [8]:
#removing special character and numbers
def rem_sp_char(words):
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]'  # defined pattern to keep
    words=re.sub(pattern, '', words)
    return words

#tokanizing and removing punctuations
def remove_punct(words):
  tokenizer = RegexpTokenizer(r'\w+')
  tokens=tokenizer.tokenize(words)
  return tokens

#lemmitization
def lemmitizar(words):
  lemmatized_words=[]
  l = WordNetLemmatizer()
  for word in words:
      token = l.lemmatize(word)
      lemmatized_words.append(token)
  return lemmatized_words

#removing stop-words
def remove_stopwords(words):
  filter_words = []
  Stopwords = stopwords.words('english')
  for word in words:
      if word not in Stopwords:
          filter_words.append(word)
  return filter_words

#converting to lowercase
def lower_case(words):
  lower_words=[]
  for word in words:
    lower_words.append(word.lower())
  return lower_words

In [9]:
#path of text files
data_path='/media/umar_visionx/Backup Plus/Active/Faizan/dataset'
test_path='/media/umar_visionx/Backup Plus/Active/Faizan/test_dataset'

In [11]:
#label encoding
labl=[]
for folder in os.listdir(data_path):
  labl.append(folder)#making the list of folder names which are labels


#encoding the labels from the list
label_index={}
for index,label in enumerate(labl):
  label_index[label]=index

label_index#label to index


{'Admin Note': 0,
 'Clinical History And Summary': 1,
 'Consult Note': 2,
 'Cover Page': 3,
 'Imaging Note': 4,
 'Insurance Authorization': 5,
 'Intake Forms': 6,
 'Lab Test': 7,
 'Other': 8,
 'Patient Profile': 9,
 'Prescriptions': 10,
 'Referral Letter': 11,
 'Requisition Form': 12}

In [13]:
#making dictionaries and lists of text and labels
def preprocessed_df(data_path):
    data_dict={}
    #txt_file_path=[]
    labels=[]
    text=[]
    encoded_labels=[]
    for clss in os.listdir(data_path):
        clss_path=os.path.join(data_path,clss)
        for txt_file in os.listdir(clss_path):
            file_path=os.path.join(clss_path,txt_file)
            #reading text file
            with open(file_path, "r") as file:
                para = file.read().rstrip('\n')#rstrip method removes trailing character based on string argument passed

            #apply preprocessing functions on para
                words = rem_sp_char(para) #removing special character and numbers
                tokens = remove_punct(words) #removing punctuations and tokanization
                tokens = lemmitizar(tokens) #lemmitization
                tokens = remove_stopwords(tokens) ##removing stop-words
                tokens = lower_case(tokens) #converting to lowercase

            #joining words with space and removing single tetter tokens
                parag=' '.join( [word for word in tokens if len(word)>1] )
            #parag=' '.join( [word for word in tokens if len(word)>1 and word.isalpha()] )#if we dont want numeric values


            #inseting data into lists and dictionaries
                text.append(parag)
            #text_file_path.append(file_path)
                labels.append(clss)
                data_dict[parag]=clss
#label encoding
    for ent in labels:
        encoded_labels.append(label_index[ent])
    


    df=pd.DataFrame()
    df['text']=text
    df['labels']=encoded_labels
    return df

In [14]:
df=preprocessed_df(data_path)
df_test=preprocessed_df(test_path)
df.head()

,text,labels
0,29 2021 10 42 alberta health services rrduser ...,0
1,30 2021 09 34 alberta health services rrduser ...,0
2,29 2021 11 03 alberta health services rrduser ...,0
3,2021 08 25 alberta health services rrduser 403...,0
4,ahs 2021 23 05 am page alberta health services...,0


In [15]:
print(len(df))
print(len(df_test))


11660
1810


In [16]:
#checking null values in dataframes
print(df.isnull().any().sum())
print(df_test.isnull().any().sum())

0
0


In [17]:
print(f"Unique Labels after Encoding:  {df.labels.unique()}")

Unique Labels after Encoding:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [18]:
#BERT base model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)#model returns all hidden-states.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df)
#val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [20]:
def encode_example(sample,max_seq_length=512):
    words=sample['text']
    label=sample['labels']
    #splitting all words of sentence in to into tokens
    tokenized_text = tokenizer.tokenize(words)
    # Truncation of token_boxes
    special_tokens_count = 2 
    if len(tokenized_text) > max_seq_length - special_tokens_count:
      tokenized_text = tokenized_text[: (max_seq_length - special_tokens_count)]#to make place available for cls and sep tokens
    # Adding CLS and SEP tokens
    input_sequence = ["[CLS]"] + tokenized_text + ["[SEP]"]
    # Map the tokens to their vocabulary indeces
    indexed_tokens = tokenizer.convert_tokens_to_ids(input_sequence)
    #padding length
    padding_length = max_seq_length - len(input_sequence)
    indexed_tokens += [0] * padding_length#zero padding to maximum length
    #as we are giving as a single sentence so mark each token as belonging to sentence 1
    segments_ids = [1] * max_seq_length
    pad_masks = [1] * len(input_sequence) + [0] * padding_length


    assert len(indexed_tokens) == max_seq_length
    assert len(segments_ids) ==max_seq_length
    assert len(pad_masks) == max_seq_length

    #converting into pytorch tensors
    tokens_tensor = torch.tensor(indexed_tokens)
    segments_tensors = torch.tensor(segments_ids)
    pad_mask_tensors=torch.tensor(pad_masks)
    label_tensors=torch.tensor(label)
    
  
    tok_tensors.append(tokens_tensor)
    seg_tensors.append(segments_tensors)
    lab_tensors.append(label_tensors)
    pad_msk_tensors.append(pad_mask_tensors)


In [22]:
def iter_df(dataset):
    global tok_tensors,seg_tensors,lab_tensors,pad_msk_tensors

    tok_tensors=[]
    seg_tensors=[]
    lab_tensors=[]
    pad_msk_tensors=[]
    dataset.map(encode_example)
    # df=pd.DataFrame()
    # df['token_tensors']=tok_tensors
    # df['segment_tensor']=seg_tensors
    # df['labels']=lab_tensors
    return tok_tensors,seg_tensors,lab_tensors,pad_msk_tensors

In [23]:
tr_token_tensor,tr_segment_tensors,tr_label_tensors,tr_pad_msk_tensors=iter_df(train_dataset)
ts_token_tensor,ts_segment_tensors,ts_label_tensorsts,ts_pad_msk_tensors=iter_df(test_dataset)

  0%|          | 0/11660 [00:00<?, ?ex/s]

  0%|          | 0/1810 [00:00<?, ?ex/s]

In [24]:
from torch.utils.data import DataLoader,Dataset
class PhelixDataset(Dataset):
    def __init__(self, token_tens,label_tens,seg_tens,pd_msk_tens):
        #super().__init__()
        self.token_tens = token_tens
        self.seg_tens=seg_tens
        self.label_tens=label_tens
        self.pd_msk_tens=pd_msk_tens

    def __len__(self):
        return len(self.token_tens)
    
    def __getitem__(self, index):       
        tokens_id = self.token_tens[index]
        segment_id = self.seg_tens[index]
        label=self.label_tens[index]
        pad_mask=self.pd_msk_tens[index]

        return tokens_id,label,segment_id,pad_mask

In [26]:
batch_size=1
train_data = PhelixDataset(tr_token_tensor,tr_label_tensors,tr_segment_tensors,tr_pad_msk_tensors)
test_data = PhelixDataset(ts_token_tensor,ts_label_tensorsts,ts_segment_tensors,ts_pad_msk_tensors)
train_dataloader = DataLoader(train_data, batch_size = batch_size,shuffle=True,num_workers=3,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size = batch_size,shuffle=False,num_workers=3,pin_memory=True)
sample = next(iter(train_dataloader))
print(sample)
# batch=next(iter(train_dataloader))
# print(batch)

[tensor([[  101,  2403, 25682,  5641,  2654,  2484, 22851,  2102,  2000, 28754,
          2683, 16932, 17788,  2575,  2487,  3931,  2013,  2092,  2740,  6786,
          6904,  2595,  3438, 26224, 16932, 17788,  2575,  2487,  2092,  2740,
          5624,  9697,  2966,  9349,  5292, 17414,  6523,  7941,  6098,  9004,
          2906,  3802,  2015,  1051,  2063,  2000, 18856, 19429,  4315, 16216,
          8840,  2140,  5776,  2171,  2966,  9349,  4205,  3231, 21650,  2184,
          2760,  2238,  6122,  6098,  4751,  2238,  2403, 25682,  2260,  4002,
         15017, 12740,  5179, 13642, 10550,  9948,  4647,  1058,  2509,  2015,
          1019,  2243,  2581,  2710,  6523,  7941,  5292, 18235,  2078,  2363,
          2238,  6400,  6523,  7941,  5292, 17414,  2023,  6523,  7941, 11333,
         13012, 18655,  6887, 20806,  2595,  9932,  4748, 10020,  3353,  9349,
          3942,  7479,  6887, 20806,  2595,  9932,  4553,   102,     0,     0,
             0,     0,     0,     0,     0,     0, 

In [27]:
for token, label,seg,pad_mask in train_dataloader:
    print(token.shape)
    print(label.shape)
    print(seg.shape)
    print(pad_mask.shape)
    break

torch.Size([1, 512])
torch.Size([1])
torch.Size([1, 512])
torch.Size([1, 512])


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [29]:
# #training the model
# global_step = 0

# #put the model in training mode
# model.train()
# ###
# train_epoch=[]
# train_acc=[]
# val_acc=[]
# curr_loss=[]
# #val_epoch=[]
# #train_losses, valid_losses,valid_acc = [], [], []


# for epoch in range(num_epochs):
#   print("Epoch:", epoch)
#   ###
#   train_epoch.append(epoch)
#   running_loss = 0.0
#   correct = 0
#   for images, labels in train_dataloader:
   
#       #moving to device
#       images = images.to(device)
#       labels = labels.to(device)

#   #     # forward pass
#       logits = model(images)
#       loss = criterion(logits,labels)
#     #   print(logits)
#     #   print(labels)
      
  
#       running_loss += loss.item()
#       predictions = logits.argmax(-1)
#       correct += (predictions == labels).float().sum()
#     #   print(loss)
#   #     # backward pass to get the gradients 
#       loss.backward()

#   #     # update
#       optimizer.step()
#       optimizer.zero_grad()
#       global_step += 1

  
#   print("Loss:", running_loss / batch_size)
#   accuracy = 100 * correct / len(df_train)
#   print("Training accuracy:", accuracy.item())
#   ###
#   train_acc.append(accuracy.item())
#   ###
#   curr_loss.append(running_loss / batch_size)
#   #saving model
#   torch.save(model.state_dict(), model_chkp_path)

# #   #checking on validation dataset
# #   #if epoch%5==0:
# #   model.eval()

# #   correct = 0
# #   for batch in val_dataloader:

# #     #moving to device
# #     images = images.to(device)
# #     labels = labels.to(device)


# #     logits = model(images)

# #     predictions = logits.argmax(-1)
# #     correct += (predictions == labels).float().sum()

# #   accuracy = 100 * correct / len(df_val)
# #   print("Validation accuracy:", accuracy.item())
# #     ###
# #   val_acc.append(accuracy.item())
    
#   print("-----------")

In [33]:
import torch
torch.cuda.empty_cache()

In [34]:
#testing GPU
!nvidia-smi

Tue Mar 15 11:14:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:65:00.0  On |                  N/A |
|  0%   38C    P2    68W / 320W |   1417MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
saved_data='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors'

In [7]:
import pickle
def embed(data_loader,train):
  #creating word embeddings
  model.eval()
  # token_labels_list=[]
  # concat_token_vect=[]#define list to append concatinated token vectors
  # doc_vectors_list=[]
  # doc_label_list=[]
  #to save data as arrays 
  token_labels_list_arr=[]
  concat_token_vect_arr=[]
  # doc_vectors_list_arr=[]
  # doc_label_list_arr=[]




  for tokens,label,seg_id,_ in tqdm(data_loader):
      if train==True:
        ft=open(os.path.join(saved_data,'train_feat.pickle'), 'ab')
        lb=open(os.path.join(saved_data,'train_label.pickle'), 'ab')
      else:
        ft=open(os.path.join(saved_data,'test_feat.pickle'), 'ab')
        lb=open(os.path.join(saved_data,'test_label.pickle'), 'ab')

      #moving to device
      label=int(label)
      tokens=tokens.to(device)
      seg_id=seg_id.to(device)
      with torch.no_grad():
        output = model(tokens,seg_id)
      #we set output_hidden_state=True in our pre trained model so, we will get the hidden states from all 13 layers at third positiob
      hidden_states = output[2]
      #For futher info please check https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
      # Concatenating tensors for all layers and creating new dim with the help of stack
      token_embed = torch.stack(hidden_states, dim=0)#token embeddings(torch.Size([13, 1, 512, 768]))
      token_embed = torch.squeeze(token_embed, dim=1)#removing batch dimension(torch.Size([13, 512, 768]))
      token_embed = token_embed.permute(1,0,2)# swapping dimensions(torch.Size([512, 13, 768]))
      #iterating for each token in whole document/sentense
      to_append_vect=[]
      to_append_vect_arr=[]
      #to_append_label=[]
      for token in token_embed:
          #token vector of size 13*768 where 13 are number of layers while we want to get only last 4 layers i.e 4*768
          #after concatination last four layers vector dimensions will be 4*768=3,072(singlre word vector length i.e per token)
          #concatinating vectors
          add1 = torch.add(token[-1], token[-2])
          add2=torch.add(token[-3], token[-4])
          full_vector=torch.add(add1,add2)
          #concatcat_vector = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
          # concat_token_vect.append(full_vector)
          #we have append 512 vectors of size 768 for each document
          # token_labels_list.append(label)
          token_labels_list_arr.append(label)
          concat_token_vect_arr.append(full_vector.cpu().numpy())

          to_append_vect_arr.append(full_vector.cpu().numpy())
          to_append_vect.append(full_vector)
          #to_append_label.append(label)
      # print(len(to_append_vect_arr))
      # print(len(to_append_vect))


      # doc_vectors_list.append(to_append_vect)
      # doc_label_list.append(label)
      # doc_vectors_list_arr.append(to_append_vect_arr)
      # doc_label_list_arr.append(label)

      # with open(os.path.join(saved_data,"train_features.pickle"),'ab') as f:
      #   pickle.dump(to_append_vect_arr,f)
      # with open(os.path.join(saved_data,"train_labels.pickle"),'ab') as f:
      #   pickle.dump(label,f)
      pickle.dump(to_append_vect_arr,ft)
      pickle.dump(label,lb)
  ft.close()
  lb.close()

  #return doc_vectors_list_arr,doc_label_list_arr

In [ ]:
# Generating Embeddings

embed(train_dataloader,True)
embed(test_dataloader,False)

In [2]:
import pickle
#loading data from pickle file
def ext_data(filepath):

    data = []
    with open(filepath, 'rb') as fr:
        try:
            while True:
                data.append(pickle.load(fr))
        except EOFError:
            pass
    return data

In [17]:
from tqdm import tqdm
train_feat_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/train_feat.pickle'
train_label_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/train_label.pickle'
test_feat_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/test_feat.pickle'
test_label_path='/media/umar_visionx/Backup Plus/Active/Faizan/saved_vectors/test_label.pickle'
doc_vectors_list_arr=ext_data(train_feat_path)
doc_label_list_arr=ext_data(train_label_path)
test_vector_list_arr=ext_data(test_feat_path)
test_label_list_arr=ext_data(test_label_path)

In [18]:
# del doc_vectors_list_arr
# del doc_label_list_arr

In [4]:
#this list have document vectors,every vector,every document have 512 token vectors and every oken vector have 3072 entries
print(len(doc_vectors_list_arr))
print(len(doc_label_list_arr))
print(len(doc_vectors_list_arr[1]))
print(len(doc_label_list_arr))
print(len(doc_vectors_list_arr[1][1]))

11660
11660
512
11660
768


In [4]:
#splitiing the data into train and validation
x_t, x_v, y_t, y_v = train_test_split(doc_vectors_list_arr, doc_label_list_arr, test_size=0.1,random_state=42,stratify=doc_label_list_arr)
print(len(x_t))
print(len(x_v))

10494
1166


In [19]:
#converting features into numpy array
x_train=[]
y_train=[]
x_val=[]
y_val=[]
x_test=[]
y_test=[]

# #train set
# for ent in x_t:
#     x_train.append(np.array(ent,dtype=np.float32))

# for ent in y_t:
#     y_train.append(np.array(ent,dtype=np.int32))

#     #validation set
# for ent in x_v:
#     x_val.append(np.array(ent,dtype=np.float32))

# for ent in y_v:
#      y_val.append(np.array(ent,dtype=np.int32))

#test set
for ent in test_vector_list_arr:
    x_test.append(np.array(ent,dtype=np.float32))

for ent in test_label_list_arr:
    y_test.append(np.array(ent,dtype=np.int32))

# print(len(x_train))
# print(len(y_train))
# print(len(x_val))
# print(len(y_val))
print(len(x_test))
print(len(y_test))

    


1810
1810


In [16]:
# del x_train
# del y_train
# del x_val
# del y_val

In [17]:
# #For more info about attention layer please see following article which is very easy and well explained
# #https://towardsdatascience.com/create-your-own-custom-attention-layer-understand-all-flavours-2201b5e8be9e
# class WordAttention(Layer):
#     def __init__(self):
#         super(WordAttention,self).__init__()
#     def build(self,input_shapes):
#         #we have one unit of MLP
#         self.w=self.add_weight(shape=(1536,1), initializer="normal")
#         self.b=self.add_weight(shape=(512,1), initializer="zeros")
#         super(WordAttention, self).build(input_shapes)
#     def call(self, x):
#         #x is inp tensor of 1536 dimensions
#         #main processing done during training
#         e = K.tanh(K.dot(x,self.w)+self.b)
#         a = K.softmax(e, axis=1)
#         output = x*a
        
#         # return the outputs. 'a' is the set of 512 attention weights
#         # the second variable is the 'attention adjusted o/p state'
#         return a, K.sum(output, axis=1)


In [18]:
# ################Runnig code###################
# class WordAttention(Layer):
#     def __init__(self):
#         super(WordAttention,self).__init__()
#     def build(self,input_shapes):
#         #we have one unit of MLP
#         self.w=self.add_weight(shape=(1536,1), initializer="normal")
#         self.b=self.add_weight(shape=(512,1), initializer="zeros")
#         super(WordAttention, self).build(input_shapes)
#     def call(self, x):
#         #x is inp of 1536 dimensions
#         #main processing done during training
#         e = K.tanh(K.dot(x,self.w)+self.b)
#         a = K.softmax(e, axis=1)
#         output = x*a
        
#         # return the outputs. 'a' is the set of 512 attention weights
#         # the second variable is the 'attention adjusted o/p state'
#         return a, K.sum(output, axis=1)

In [19]:
# pip install -U gast==0.2.2

In [7]:
class AttentionLayer(Layer):
    """
    Source: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
            https://humboldt-wi.github.io/blog/research/information_systems_1819/group5_han/
   
    """


    def __init__(self,attention_dim=768,return_coefficients=False,**kwargs):
        # Initializer
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.return_coefficients = return_coefficients
        self.attention_dim = attention_dim
        self.supports_masking = True
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # print(input_shape[0])
        # print(input_shape[-1])
        # print(input_shape[1])
        # Builds all weights
        # u = context vector,W = Weight matrix, b = bias vector
        assert len(input_shape) == 3
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.W = K.variable(self.init((int(input_shape[-1]), self.attention_dim)),name='W')
        
        self._trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], int(input_shape[-1])), (input_shape[0], int(input_shape[-1]), 1)]
        else:
            return input_shape[0], int(input_shape[-1])

    # def get_config(self):

    #     config = super().get_config().copy()
    #     config.update({
    #         'attention_dim ': self.attention_dim ,
    #         'return_coefficients': self.return_coefficients,
    #     })
    #     return config

    def get_config(self):#to avoiding saving problem.check(https://stackoverflow.com/questions/50837728/valueerror-unknown-layer-capsulelayer and    
        #https://stackoverflow.com/questions/58678836/notimplementederror-layers-with-arguments-in-init-must-override-get-conf)
        # For serialization with 'custom_objects'
        config = super().get_config()
        config['attention_dim'] = self.attention_dim
        config['return_coefficients'] = self.return_coefficients
        return config


In [ ]:
import tensorflow
import tensorflow as tf
# embedding_matrix = np.random.random((VOCAB_SIZE, EMBEDDING_DIM))
# max_sentenses=1
# embedding_layer = Embedding(VOCAB_SIZE,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=False,weights=[embedding_matrix],name='word_embedding')
MAX_SEQUENCE_LENGTH=512
EMBEDDING_DIM=768
max_sentenses=1

# Words  attention
emb_input = Input(shape=(MAX_SEQUENCE_LENGTH,EMBEDDING_DIM), dtype='float32',name='embeddings_input')
# #word_sequences = embedding_layer(word_input)
gru_word = Bidirectional(GRU(768, return_sequences=True),name='word_GRU')(emb_input)#by default if we set treturn_sequence=False it will
#return only last words hidden state is returm only as the output having vector dimensions=2*GRU units 
#for bidirectional
# word_dense = Dense(1536, activation='relu', name='word_dense')(gru_out) 
word_attention,word_coff= AttentionLayer(EMBEDDING_DIM,return_coefficients=True,name='word_attention')(gru_word)

# dense_out = Dense(512,activation='relu', name='word_dense')(word_attention)#we may use different number of units
# preds = Dense(13, activation='softmax',name='output')(dense_out)
# model = Model(emb_input, preds)
WordEnc = Model(inputs = emb_input,outputs = word_attention)#word encoder



#Sentence Attention
sentence_inp=Input(shape=(max_sentenses, MAX_SEQUENCE_LENGTH,EMBEDDING_DIM), dtype='float32')
sent_encoder = TimeDistributed(WordEnc,name='sent_encd')(sentence_inp)
gru_sentense = Bidirectional(GRU(100, return_sequences=True),name='sentense_GRU')(sent_encoder)
sentense_attention,sentense_coff= AttentionLayer(EMBEDDING_DIM,True,name='sentense_attention')(gru_sentense)
dense_out = Dense(512,activation='relu', name='sentense_dense')(sentense_attention)#we may use different number of units
dropout = Dropout(0.4,name='dropout')(dense_out)
preds = Dense(13, activation='softmax',name='output')(dropout)
model = Model(sentence_inp, preds)


In [45]:

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
print(WordEnc.summary())
print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings_input (InputLaye  [(None, 512, 768)]       0         
 r)                                                              
                                                                 
 word_GRU (Bidirectional)    (None, 512, 1536)         7087104   
                                                                 
 word_attention (AttentionLa  [(None, 1536),           1181184   
 yer)                         (None, 512, 1)]                    
                                                                 
Total params: 8,268,288
Trainable params: 8,268,288
Non-trainable params: 0
_________________________________________________________________
None
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        

In [20]:
#Adding dimension into training and test data and converting into numpy array
# x_tr = np.array([x_train])
# y_tr = np.array(y_train)
x_ts=np.array([x_test])
y_ts=np.array(y_test)
#Reshaping inputs
# print(x_tr.shape)
# print(y_tr.shape)
print(x_ts.shape)
print(y_ts.shape)
# x_trn=np.transpose(x_tr, (1, 0, 2,3))
x_tst=np.transpose(x_ts, (1, 0, 2,3))
# print(x_trn.shape)
print(x_tst.shape)

(1, 1810, 512, 768)
(1810,)
(1810, 1, 512, 768)


In [8]:
#data loader
from tensorflow.keras.utils import Sequence
import math
class DataGenerator(Sequence):

    def __init__(self, x_train, y_train,batch_size):
        self.x, self.y = x_train, y_train
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)#ValueError: axes don't match array

    def __getitem__(self, idx):
     
        x_tr = np.array([self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]])#we also added an extra dimension according to our requirement
        y_tr = np.array(self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size])

        x_trn=np.transpose(x_tr, (1, 0, 2,3))#reshape the input

        return x_trn,y_tr

In [9]:
train_generator=DataGenerator(x_train, y_train,32)
val_generator=DataGenerator(x_val, y_val,32)
#test_generator = DataGenerator(x_test,y_test,32)

In [13]:
#checking the total samples in each class(label)
from collections import Counter
print(f"labels: {Counter(doc_label_list_arr).keys()}")
print(f"samples in each label: {Counter(doc_label_list_arr).values()}")


labels: dict_keys([5, 2, 1, 7, 0, 11, 8, 3, 4, 12, 10, 9, 6])
samples in each label: dict_values([969, 1438, 1045, 780, 588, 2363, 982, 1542, 491, 1312, 55, 72, 23])


In [ ]:
#making class weights dictionary (our classes are imbalance)
#getting class having maximum samples
class_samples={}
train_text='/media/umar_visionx/Backup Plus/Active/Faizan/textrect_dataset/my_model.h5'
for folder in os.listdir(train_text):
    folder_path=os.path.join(train_text,folder)
    class_samples[folder]=len(os.listdir(folder_path))
print(f'max samples in class ( {max(class_samples, key=class_samples.get)}={class_samples[max(class_samples, key=class_samples.get)]})')
max_samples=int(class_samples[max(class_samples, key=class_samples.get)])
print('***********')
#getting class weights
weights={}
for folder in os.listdir(train_text):
    folder_path=os.path.join(train_text,folder)
    weights[folder]=round(max_samples/int(len(os.listdir(folder_path))),2)
print(f'class weights: {weights}')
print("********")
#changing the names of keys in class weight dictionary
class_weight={}
for (keys1,values1),(keys2,values2) in zip(weights.items(),label_index.items()):

    class_weight[int(values2)]=values1

print(f'class weights: {class_weight}')

In [2]:
#model training
checkpoint_path='/media/umar_visionx/Backup Plus/Active/Faizan/model_keras/my_model.h5'

loss_list=[]
#loss_list.append(history.history['val_loss'][0])
history=model.fit(train_generator,validation_data=val_generator, epochs=1,class_weight=class_weight,shuffle=True)
loss_list.append(history.history['val_loss'][0])
for i in range(9):
    history=model.fit(train_generator,validation_data=val_generator, epochs=1,class_weight=class_weight,shuffle=True)
    loss=history.history['val_loss'][0]
    if loss<loss_list[-1]:
        #saving the model
        model.save(checkpoint_path)
        loss_list.append(history.history['val_loss'][0])

In [10]:
# #saving the model
# model.save('/media/umar_visionx/Backup Plus/Active/Faizan/model_keras/')

In [33]:
model = tf.keras.models.load_model('/media/umar_visionx/Backup Plus/Active/Faizan/model_keras/')
model.summary()

2022-03-21 15:03:36.486999: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'while' has 11 outputs but the _output_shapes attribute specifies shapes for 37 outputs. Output shapes may be inaccurate.
2022-03-21 15:03:37.061754: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'while' has 11 outputs but the _output_shapes attribute specifies shapes for 37 outputs. Output shapes may be inaccurate.
2022-03-21 15:03:37.073573: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'model/word_GRU/forward_gru/PartitionedCall' has 4 outputs but the _output_shapes attribute specifies shapes for 41 outputs. Output shapes may be inaccurate.
2022-03-21 15:03:37.073811: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'model/word_GRU/backward_gru/PartitionedCall' has 4 outputs but the _output_shapes attribute specifies shapes for 42 outputs. Output shapes may be inaccurate.


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 512, 768)]     0         
                                                                 
 sent_encd (TimeDistributed)  (None, 1, 1536)          8268288   
                                                                 
 sentense_GRU (Bidirectional  (None, 1, 200)           982800    
 )                                                               
                                                                 
 sentense_attention (Attenti  [(None, 200),            155136    
 onLayer)                     (None, 1, 1)]                      
                                                                 
 sentense_dense (Dense)      (None, 512)               102912    
                                                                 
 dropout (Dropout)           (None, 512)               0   

In [34]:
history.history

{'loss': [0.09565596282482147],
 'acc': [0.9690299034118652],
 'val_loss': [0.44553402066230774],
 'val_acc': [0.8936535120010376]}

In [35]:
score = model.evaluate(x_tst, y_ts, verbose = 1) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

57/57 [==============================] - 83s 1s/step - loss: 1.4675 - acc: 0.7094
Test loss: 1.4675407409667969
Test accuracy: 0.709392249584198


In [26]:
predictions=model.predict(x_tst)

In [63]:
y_ts[1:5]

array([0, 0, 0, 0], dtype=int32)

In [64]:
predictions[1:5]

array([[1.0391946e-01, 1.6219815e-02, 1.3022219e-01, 7.7651935e-03,
        1.2847678e-01, 3.9892595e-02, 9.1801547e-03, 3.1454569e-01,
        7.9592057e-02, 4.2604249e-02, 4.8336651e-02, 5.1275589e-02,
        2.7969463e-02],
       [1.5764245e-01, 8.7256664e-03, 7.6922365e-02, 4.3933887e-02,
        1.3934281e-01, 3.5013333e-02, 6.4677638e-03, 2.1565320e-01,
        7.8749344e-02, 3.4783963e-02, 8.4994912e-02, 9.8145939e-02,
        1.9624410e-02],
       [1.4916790e-01, 5.2206498e-05, 1.7044856e-03, 8.1145996e-01,
        5.2056927e-04, 3.2123746e-03, 9.3613904e-05, 1.6701395e-04,
        1.5591676e-02, 6.3124549e-04, 2.0211812e-03, 1.4666222e-02,
        7.1141985e-04],
       [1.4672674e-01, 3.7247868e-05, 1.3714962e-03, 8.2115197e-01,
        4.8205562e-04, 3.4428122e-03, 8.3032603e-05, 1.4420917e-04,
        1.3295571e-02, 5.0027523e-04, 1.4545802e-03, 1.0700453e-02,
        6.0947338e-04]], dtype=float32)

In [27]:
pred = np.argmax(predictions, axis = 1)

print(pred) 


[ 0  0  0 ... 12 12 12]


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_ts,pred))

              precision    recall  f1-score   support

           0       0.77      0.59      0.67        83
           1       0.62      0.71      0.66       336
           2       0.65      0.73      0.69       369
           3       0.93      0.96      0.94       179
           4       0.87      0.54      0.67        63
           5       0.54      0.23      0.32        31
           6       0.00      0.00      0.00         8
           7       0.80      0.83      0.82       181
           8       0.62      0.51      0.56       266
           9       0.79      0.54      0.64        41
          10       1.00      0.22      0.36         9
          11       0.75      0.93      0.83       169
          12       0.95      0.84      0.89        75

    accuracy                           0.72      1810
   macro avg       0.71      0.59      0.62      1810
weighted avg       0.72      0.72      0.71      1810

